In [1]:
import pandas as pd
from topic_modelling import get_topic_model

df = pd.read_csv('data/bills_18_23.csv', index_col=0)

# Remove rows with Referred to committee
df = df[~df['Bill Description'].str.contains('Referred to committee')]
df = df[~df['Bill Description'].str.contains('Introduced ')]
df = df[~df['Bill Description'].str.contains('Dead/Failed')]
df = df[~df['Bill Description'].str.contains('Session ended')]
df = df[~df['Bill Description'].str.contains('Committee')]
df = df[~df['Bill Description'].str.contains('Postponed Indefinitely ')]
df = df[~df['Bill Description'].str.contains('Public Hearing Completed ')]
df = df[~df['Bill Description'].str.contains('Hearing Scheduled ')]
df = df[~df['Bill Description'].str.contains('Hearing scheduled ')]
df = df[~df['Bill Description'].str.contains('Filed for bills not passed ')]
df = df[~df['Bill Description'].str.contains('Crossed Over to Senate ')]
df = df[~df['Bill Description'].str.contains('Signed/Enacted by ')]
df = df[~df['Bill Description'].str.contains('Hearing Conducted')]
df = df[~df['Bill Description'].str.contains('Subcommittee')]
df = df[~df['Bill Description'].str.contains('Passed by')]
df = df[~df['Bill Description'].str.contains('Read Second Time')]
df = df[~df['Bill Description'].str.contains('Carryover')]
df = df[~df['Bill Description'].str.contains('carryover')]
df = df[~df['Bill Description'].str.contains('referred to committee')]
df = df[~df['Bill Description'].str.contains('Removed')]
df = df[~df['Bill Description'].str.contains('Second reading')]
df = df[~df['Bill Description'].str.contains('Pub. Ch.')]
df = df[~df['Bill Description'].str.contains('Reading')]
df = df[~df['Bill Description'].str.contains('Legislate')]
df = df[~df['Bill Description'].str.contains('Laid on table')]
df = df[~df['Bill Description'].str.contains('EDC Referral')]
df = df[~df['Bill Description'].str.contains('Site Information')]

/Users/yangxinyuxie/Desktop/anti-trans-bills/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# only include bills with descriptions whose length is greater than 10
df = df[df['Bill Description'].str.len() > 10]

In [3]:
df['Bill Type'].unique()

array(['Youth Athletics', 'Healthcare', 'Public Facilities', 'ID Updates',
       'Religious Freedom', 'Other', 'Schools / Education',
       'Nondiscrimination Protections', 'Drag Performance'], dtype=object)

In [4]:
category = 'Drag Performance'

docs, embeddings, topic_model, topic_df, info_tab = get_topic_model(df, category)

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.33s/it]
2024-03-17 17:23:12,021 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-17 17:23:15,274 - BERTopic - Dimensionality - Completed ✓
2024-03-17 17:23:15,274 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-17 17:23:15,277 - BERTopic - Cluster - Completed ✓
2024-03-17 17:23:15,278 - BERTopic - Representation - Extracting topics from clusters using representation models.
  0%|          | 0/3 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 3/3 [00:02<00:00,  1.16it/s]
2024-03-17 17:23:19,187 - BERTopic - Representation - Completed ✓


In [5]:
fig = topic_model.visualize_hierarchy(custom_labels=True)
# save the plotly figure as png
fig.write_image(f"data/{category}/hierarchical_topic_model.png")
fig

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
from umap import UMAP
reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.1, metric='cosine').fit_transform(embeddings)

In [7]:
# Visualize the documents in 2-dimensional space and show the titles on hover instead of the abstracts
# NOTE: You can hide the hover with `hide_document_hover=True` which is especially helpful if you have a large dataset
fig = topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, custom_labels=True)
fig.write_image(f"data/{category}/clusering_visualization.png")
fig